# Introducing tf.estimator.train_and_evaluate()
**Learning Objectives**
- Introduce new type of input function (`serving_input_reciever_fn()`) which supports remote access to our model via REST API
- Use the `tf.estimator.train_and_evaluate()` method to periodically evaluate *during* training
- Practice using TensorBoard to visualize training and evaluation loss curves

Run the following cell and reset the session if needed:

In [ ]:
!pip freeze | grep tensorflow==1.12.0 || pip install tensorflow==1.12.0

In [ ]:
import tensorflow as tf
import shutil
assert tf.__version__ == "1.12.0"

## 1) Train and Evaluate Input Functions


In [ ]:
CSV_COLUMN_NAMES = ['fare_amount','dayofweek','hourofday','pickuplon','pickuplat','dropofflon','dropofflat']
CSV_DEFAULTS = [[0.0],[1],[0],[-74.0], [40.0], [-74.0], [40.7]]

In [ ]:
def parse_row(row):
    fields = tf.decode_csv(row, record_defaults=CSV_DEFAULTS)
    features = dict(zip(CSV_COLUMN_NAMES, fields))
    label = features.pop('fare_amount')
    return features, label


def read_dataset(csv_path):
    dataset = tf.data.TextLineDataset(csv_path).skip(1) # skip header
    dataset = dataset.map(parse_row)
    return dataset


def train_input_fn(csv_path, batch_size=128):
    dataset = read_dataset(csv_path)
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset


def eval_input_fn(csv_path, batch_size=128):
    dataset = read_dataset(csv_path)
    dataset = dataset.batch(batch_size)
    return dataset

## 2) Feature Columns

Same as before

In [ ]:
FEATURE_NAMES = CSV_COLUMN_NAMES[1:] # all but first column

feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURE_NAMES]
feature_cols

## 3) Serving Input Receiver Function 

In a prior notebook we used the `estimator.predict()` function to get taxifare predictions. This worked fine because we had done our model training on the same machine. 

However in a production setting this won't usually be the case. Our clients may be remote web servers, mobile apps and more. Instead of having to ship our model files to every client, it would be better to host our model in one place, and make it remotely accesible for prediction requests using a REST API.

The TensorFlow solution for this is a project called [TF Serving](https://www.tensorflow.org/serving/), which is part of the larger [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx/) platform that extends TensorFlow for production environments. 

The interface between TensorFlow and TF Serving is a `serving_input_receiver_fn()`. It has two jobs:
- To add `tf.placeholder`s to the graph to specify what type of tensors TF Serving should recieve during inference requests.  The placeholders are specified as a dictionary object
- To add any additional ops needed to convert data from the client into the tensors expected by the model.

The function must return a `tf.estimator.export.ServingInputReceiver` object, which packages the placeholders and the neccesary transformations together.

### Exercise 1

In the cell below, implement a `serving_input_receiver_fn` function that returns an instance of
```python
tf.estimator.export.ServingInputReceiver(features, receiver_tensors)
```
where `receiver_tensors` is a dictionary describing the JSON object received by the Cloud ML Engine API,
and is a dictionary `features` that has the structure as the feature dictionary accepted by our estimator.

Here we keep things simple by assuming that the API receives a JSON object that has already the correct structure
(i.e. `features = receiver_tensors`):

In [ ]:
def serving_input_receiver_fn():
  # TODO
  return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

## 4) Monitoring with TensorBoard 

[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is a web UI that allows us to visualize various aspects of our model, including the training and evaluation loss curves. Although you won't see the loss curves yet, it is best to launch TensorBoard *before* you start training so that you may see them update during a long running training process.

*Warning:* There is an issue with DataLab that causes TensorBoard to only work correctly the first time it's launched per session. So if you have issues try resetting the kernel and launching TensorBoard again


In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('taxi_trained')

## 5) Train and Evaluate

One issue with the previous notebooks is we only evaluate on our validation data once training is complete. This means we can't tell at what point overfitting began. What we really want is to evaluate at specified intervals *during* the training phase.

The Estimator API way of doing this is to replace `estimator.train()` and `estimator.evaluate()` with `estimator.train_and_evaluate()`. This causes an evaluation to be done after every training checkpoint. However by default Tensorflow only checkpoints once every  10 minutes. Since this is less than the length of our total training we'd end up with the same behavior as before which is just one evaluation at the end of training. 

To remedy this we speciy in the `tf.estimator.RunConfig()` that TensorFlow should checkpoint every 100 steps.

The default evaluation metric `average_loss` is MSE, but we want RMSE. Previously we just took the square root of the final `average_loss`. However it would be better if we could calculate RMSE not just at the end, but for every intermediate checkpoint and plot the change over time in TensorBoard. [`tf.contrib.estimator.add_metrics()`](https://www.tensorflow.org/api_docs/python/tf/contrib/estimator/add_metrics) allows us to do this. We wrap our estimator with it, and provide a custom evaluation function.

`train_and_evaluate()` also allows us to use our `serving_input_receiver_fn()` to export our models in the SavedModel format required by TF Serving.

*Note: Training will be slower than the last notebook because we are now evaluating after every 100 train steps. Previously we didn't evaluate until training finished.*

### Exercise 2

In the cell below, create a instance of `tf.estimator.RunConfig` named `config`. Pass to its 
constructor information concerning:
  - the directory where we want the trained model and its checkpoints to be saved
  - the random seed which we want to be set to 1
  - the cadence at which we want the model to create checkpoints (every 100 steps)

In [ ]:
config = #TODO

### Exercise 3

In the cell below, create a `DNNRegressor` model with two layers of 10 neurons each using 
the `RunConfig` instance and the `feature_cols` list you just created.

Do not pass the model directory directly to the estimator constructor, since that info should
already be wrapped into the `RunConfig` instance.


In [ ]:
model = #TODO

### Exercise 4

Implement a `my_rmse` function that
- takes as input a tensor of `labels` and a tensor of `predictions`
- returns a dictionary with the single key `rmse` and with value the root mean square error between the labels and the predictions

**Hint:** Check `tf.metrics.root_mean_squared_error`. You may also need to use `tf.squeeze`

In [ ]:
def my_rmse(labels, predictions):
  #TODO

Run the following cell to add the custom metric you defined above to the `model`:

In [ ]:
model = tf.contrib.estimator.add_metrics(model, my_rmse)  

### Exercise 5

Create an instance of a `tf.estimator.TrainSpec` using the `train_input_fn` defined at the top of this file and
with a `max_steps` of 500. 

The training data should be loaded from `./taxi-train.csv`.

**Hint:** You may need to use a `lambda` function to pass the
training input function correctly.

In [ ]:
train_spec = #TODO

Create an exporter using the `serving_input_receiver_fn` defined at the beginning of this notebook.

You want to export the trained model and its checkpoints in the './exporter' subdirectory.

Use `tf.estimator.FinalExport` to create the exporter intance. 

**Note:** You may alternatively use `tf.estimator.BestExporter`
  to export at every checkpoint that has lower loss than the previous checkpoint, instead
  of exporting only the last checkpoint.

In [ ]:
exporter = #TODO

In the cell below, create an instance of an `EvalSpec` to which you specify
that 
- the data should be loaded from `/.taxi-valid.csv` during evaluation (use the correct input function!)
- the exporter you defined above should be used
- the first evaluation should start after 1 second of training
- and then be repeated every 1 second

**Note:** We use the checkpoint setting above because we want to evaluate after every checkpoint.
As long as checkpoints are > 1 sec apart this ensures the throttling never kicks in.

In [ ]:
eval_spec = #TODO

Run the following cell to start the training and evaluation as you specified them above:

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO) 
shutil.rmtree(OUTDIR, ignore_errors = True) 

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

## 6) Inspect Export Folder

Now in the output directory, in addition to the checkpoint files, you'll see a subfolder called 'export'. This contains one or models in the SavedModel format which is compatible with TF Serving. In the next notebook we will deploy the SavedModel behind a production grade REST API.

In [ ]:
!ls -R taxi_trained/export

## 7) Cleanup

In [ ]:
if len(TensorBoard.list())>0:
  [TensorBoard().stop(pid)for pid in TensorBoard.list()['pid']]
else: print('No TensorBoard instances to stop')

## Challenge Exercise

Modify your solution to the challenge exercise in c_dataset.ipynb appropriately.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License